In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!ls ../input/tabular-playground-series-mar-2021

In [ ]:
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.preprocessing import LabelEncoder, StandardScaler
import random
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-mar-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-mar-2021/test.csv")
sample = pd.read_csv("../input/tabular-playground-series-mar-2021/sample_submission.csv")

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
la = LabelEncoder()
for i in train.columns:
    if train[i].dtype == 'object':
        train[i] = la.fit_transform(train[i])

In [ ]:
la = LabelEncoder()
for i in test.columns:
    if test[i].dtype == 'object':
        test[i] = la.fit_transform(test[i])

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
print(train['target'].value_counts(), end = "\t")
plt.figure(figsize = (8,5))
sns.countplot(x = train['target'])
plt.xlabel('Target', size = 12)
plt.ylabel('Count', size = 12)
plt.title('Distribution in target column', size = 12)

**Target column is imbalanced**

In [ ]:
cor = train.corr()
rel = cor['target'].sort_values(ascending = False)
rel

In [ ]:
l = []
for i in range (len(rel)):
    if rel[i] > 0:
        l.append(rel.index[i])
l

In [ ]:
y = train.loc[:, 'target']
y.head()

In [ ]:
l.remove('target')
x = train.loc[:,l]
# x.drop('target', inplace = True, axis = 1)
x.head()

In [ ]:
col = x.columns
std = StandardScaler()
x = std.fit_transform(x)
x = pd.DataFrame(data = x, columns = col)
x.head()

In [ ]:
x.describe()

In [ ]:
print(train['target'].value_counts(), end = "\t")
plt.figure(figsize = (8,5))
sns.countplot(x = train['target'])
plt.xlabel('Target', size = 12)
plt.ylabel('Count', size = 12)
plt.title('Distribution in target column before resampling', size = 12)

In [ ]:
over = RandomOverSampler()
x, y = over.fit_resample(x, y)

In [ ]:
print(y.value_counts(), end = "\t")
plt.figure(figsize = (8,5))
sns.countplot(x = y)
plt.xlabel('Target', size = 12)
plt.ylabel('Count', size = 12)
plt.title('Distribution in target column after resampling', size = 12)

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.25, random_state = 42)

In [ ]:
rcla = RandomForestClassifier(n_estimators = 250)
rcla.fit(xtrain, ytrain)

In [ ]:
predicted = rcla.predict(xtrain)
print("Accuracy using Random Forest on training data is {} %".format(accuracy_score(predicted, ytrain)*100))

In [ ]:
predicted = rcla.predict(xtest)
print("Accuracy using Random Forest testing data is {} %".format(accuracy_score(predicted, ytest)*100))

In [ ]:
dcla = DecisionTreeClassifier()
dcla.fit(xtrain, ytrain)

In [ ]:
predicted = dcla.predict(xtrain)
print("Accuracy using Random Forest on training data is {} %".format(accuracy_score(predicted, ytrain)*100))

In [ ]:
predicted = dcla.predict(xtest)
print("Accuracy using Random Forest testing data is {} %".format(accuracy_score(predicted, ytest)*100))

# Tetsing data

In [ ]:
x_test = test.loc[:,l]
# x.drop('target', inplace = True, axis = 1)
x_test.head()

In [ ]:
col = x_test.columns
std = StandardScaler()
x_test = std.fit_transform(x_test)
x_test = pd.DataFrame(data = x_test, columns = col)
x_test.head()

In [ ]:
predicted = rcla.predict(x_test)
predicted

In [ ]:
sample.head()

In [ ]:
submit = pd.DataFrame({'id':test['id'], 'target':predicted})
submit.head()

In [ ]:
submit.to_csv('submission1', index = False)